In [ ]:
import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import PIL
from PIL import Image, ImageOps
import cv2
from tqdm import tqdm_notebook
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import torch.utils.data as utils
import torchvision.models as models

import torchvision
from torchvision import transforms

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")
IMG_SIZE = 224

In [ ]:
pretrained_model = torchvision.models.densenet161(pretrained=False, num_classes=5)

In [ ]:
data_dir = '../input/aptos2019-blindness-detection/'
train_dir = data_dir + '/train_images/'
test_dir = data_dir + '/test_images/'

In [ ]:
df_train = pd.read_csv(data_dir+"train.csv")
df_test = pd.read_csv(data_dir+"test.csv")
df_test['diagnosis'] = -1

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
def crop_image_from_gray(img,tol=7):
    if img.ndim ==2:
        mask = img>tol
        return img[np.ix_(mask.any(1),mask.any(0))]
    elif img.ndim==3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img>tol
        
        check_shape = img[:,:,0][np.ix_(mask.any(1),mask.any(0))].shape[0]
        if (check_shape == 0): # image is too dark so that we crop out everything,
            return img # return original image
        else:
            img1=img[:,:,0][np.ix_(mask.any(1),mask.any(0))]
            img2=img[:,:,1][np.ix_(mask.any(1),mask.any(0))]
            img3=img[:,:,2][np.ix_(mask.any(1),mask.any(0))]
            img = np.stack([img1,img2,img3],axis=-1)
        return img

In [ ]:
def load_ben_color(path, sigmaX=10):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = crop_image_from_gray(image)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image=cv2.addWeighted ( image,4, cv2.GaussianBlur( image , (0,0) , sigmaX) ,-4 ,128)
        
    return image

In [ ]:
class ImageData(Dataset):
    def __init__(self, df, data_dir, transform):
        super().__init__()
        self.df = df.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name,label = self.df[index]
        
        img_path = os.path.join(self.data_dir, img_name+'.png')
        image = load_ben_color(img_path,sigmaX=10)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
data_transf = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.RandomRotation((-180, 180)),
    torchvision.transforms.RandomHorizontalFlip(p=0.4),
    torchvision.transforms.RandomVerticalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

transforms_valid = torchvision.transforms.Compose([
    torchvision.transforms.ToPILImage(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
def train_one_fold(i_fold, model, loss_func, optimizer, train_loader, valid_loader, n_epochs):
    best_loss = float('+inf')
    train_fold_results = []

    for epoch in range(n_epochs):
        train_start = time.time()

        print('  Epoch {}/{}'.format(epoch + 1, n_epochs))
        print('  ' + ('-' * 20))

        model.train()
        tr_loss = 0
        train_kappa = []
        optimizer.zero_grad()
        for ii, (data, target) in tqdm_notebook(enumerate(train_loader), total=len(train_loader)):

            images = data.to('cuda', dtype = torch.float)  
#             target = target.unsqueeze(1)
            labels = target.to('cuda', dtype = torch.float)  
            
            labels = labels.view(-1,1)
            optimizer.zero_grad()
            
            
            with torch.set_grad_enabled(True):
                output = model(images)   
                
                loss = loss_func(output, labels)
                loss.backward()

                optimizer.step()
            
            tr_loss += loss.item()
            
            y_actual = labels.data.cpu().numpy()
            y_pred = output[:,-1].detach().cpu().numpy()
            kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            train_kappa.append(kappa)
        
        train_kappa_epoch = np.mean(train_kappa)

        train_end = time.time()
            
        val_start = time.time()
        # Validate
        model.eval()
        
        val_loss = 0
        val_preds = None
        val_labels = None
        val_kappa = []
        
        for ii, (data, target) in tqdm_notebook(enumerate(valid_loader), total=len(valid_loader)):


            images = data.to('cuda', dtype = torch.float)
            labels = target.to('cuda', dtype = torch.float)
            labels = labels.view(-1,1)

            with torch.no_grad():
                outputs = model(images)
                
                loss = loss_func(outputs, labels)
                val_loss += loss.item()

            y_actual = labels.data.cpu().numpy()
            y_pred = outputs[:,-1].detach().cpu().numpy()
            kappa = cohen_kappa_score(y_actual, y_pred.round(), weights='quadratic')
            val_kappa.append(kappa)

           
        val_kappa_epoch = np.mean(val_kappa)
        val_end = time.time()
        train_loss = tr_loss/len(train_loader)
        valid_loss = val_loss/len(valid_loader)
        
        print('Fold: {}, Epoch: {}, Train duration: {:.6f}, Valid duration: {:.6f}, Train Loss: {:.6f}, Valid Loss: {:.6f}, Train Kappa: {:.4f}, Valid Kappa: {:.4f}'.format(i_fold+1, epoch+1, train_end - train_start, val_end - val_start, train_loss, valid_loss, train_kappa_epoch, val_kappa_epoch))
        if best_loss > valid_loss:
            best_loss = valid_loss
            torch.save(model.state_dict(), '../input/aptos-checkpoints/aptos_3_pretrained_model_161_{}_fold_{}_epoch.pth'.format(i_fold, epoch))
    return train_loss, valid_loss, train_kappa_epoch, val_kappa_epoch

In [ ]:
N_FOLDS = 2
BATCH_SIZE = 32
N_EPOCHS = 15

folds = StratifiedKFold(n_splits=N_FOLDS, shuffle=True)

In [ ]:
state_dict = torch.load('../input/aptos-checkpoints/aptos_3_pretrained_model_161_1_fold_0_epoch.pth', map_location=lambda storage, loc: storage)
pretrained_model.load_state_dict(state_dict)

In [ ]:
test_data = ImageData(df = df_test, data_dir = test_dir, transform = transforms_valid)
test_loader = DataLoader(test_data, batch_size=4, num_workers=4, shuffle=False)

In [ ]:
 def round_off_preds(preds, coef=[0.5, 1.5, 2.5, 3.5]):
    for i, pred in enumerate(preds):
            if pred < coef[0]:
                preds[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                preds[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                preds[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                preds[i] = 3
            else:
                preds[i] = 4
    return preds

In [ ]:
def predict(model, testloader):
    '''Function used to make predictions on the test set'''
    model.eval()
    preds = []
    for batch_i, (data, target) in tqdm_notebook(enumerate(testloader), total=len(testloader)):
        data, target = data, target
        output = model(data)
        pr = output.detach().cpu().numpy()
        for item in pr:
            preds.append(item.mean())
            
    return preds

In [ ]:
test_pred = round_off_preds(predict(pretrained_model, test_loader))

In [ ]:
df_test['diagnosis'] = test_pred
df_test.to_csv('submission.csv',index=False)
df_test